Usamos los datos de la JODI Oil World Database (Joint Organisations Data Initiative [JODI], 2018). Que se obtiene de aqui: https://www.jodidata.org/oil/database/data-downloads.aspx

Fecha de descarga: 1 septiembre de 2020





In [28]:
import pandas as pd
monthly = pd.read_csv("../Databases/JODI DB/world_Primary_CSV_NewFormat.csv")
monthly_second = pd.read_csv("../Databases/JODI DB/world_Secondary_CSV_NewFormat.csv")

monthly = pd.DataFrame(monthly_first)
monthly_second = pd.DataFrame(monthly_second)

# As the DB has country names referd by alpha-2-code I need to use pycountry to get actual names
#pip install pycountry
import pycountry

In [2]:

# List of countries we are interested in
# Netos = ['Albania','Angola','Saudi Arabia','Algeria','Argentina','Benin','Azerbaijan','Bolivia (Plur. State of)','Brazil','Brunei Darussalam','Cameroon','Canada','China','Colombia','Congo','Denmark','Ecuador','Egypt','United Arab Emirates','Estonia','Gabon','Indonesia','Georgia','Ghana','Guatemala','Iran (Islamic Rep. of)','Iraq','Kazakhstan','Kyrgyzstan','Kuwait','Libya','Malaysia','Mexico','Mongolia','Myanmar','Nigeria','Norway','Oman','Qatar','United Kingdom','Dem. Rep. of the Congo','Russian Federation','Trinidad and Tobago','Sudan','South Sudan','Tunisia','Turkmenistan','Viet Nam','Venezuela (Bolivar. Rep.)','Yemen']

#While passing the code below we find that names from countries as Bolivia, Iran, DR Congo and Venezuela had different names so we have to manually addapt the names to "Bolivia, Plurinational State of", "Iran, Islamic Republic of", 'Congo, The Democratic Republic of the' and "Venezuela, Bolivarian Republic of"
netos = ['Albania','Angola','Saudi Arabia','Algeria','Argentina','Benin','Azerbaijan','Bolivia, Plurinational State of','Brazil','Brunei Darussalam','Cameroon','Canada','China','Colombia','Congo','Denmark','Ecuador','Egypt','United Arab Emirates','Estonia','Gabon','Indonesia','Georgia','Ghana','Guatemala','Iran, Islamic Republic of','Iraq','Kazakhstan','Kyrgyzstan','Kuwait','Libya','Malaysia','Mexico','Mongolia','Myanmar','Nigeria','Norway','Oman','Qatar','United Kingdom','Congo, The Democratic Republic of the','Russian Federation','Trinidad and Tobago','Sudan','South Sudan','Tunisia','Turkmenistan','Viet Nam','Venezuela, Bolivarian Republic of','Yemen']

#Create the same list with the alpha_2 code
a_codes = []
for x in netos:
    try:
        country = pycountry.countries.get(name=x)
        a_codes.append(country.alpha_2)
    except AttributeError:
        print(f'Theres an error with {x}')
        pass

#Check lenght
len(netos) - len(a_codes)

0

In [3]:
#### Identify which countries of interest are missing from the data set
# available = monthly["REF_AREA"].unique().tolist()

# for x in a_codes:
#     if x not in available:
#         country = pycountry.countries.get(alpha_2=x)
#         print(country.name)


#### On this data set there are no data for this countries:
# Benin
# Cameroon
# Congo
# Ghana
# Kyrgyzstan
# Mongolia
# Congo, The Democratic Republic of the
# South Sudan
# Turkmenistan

Benin
Cameroon
Congo
Ghana
Kyrgyzstan
Mongolia
Congo, The Democratic Republic of the
South Sudan
Turkmenistan


In [39]:
#Clean the DB from missing values
monthly = monthly.dropna(how="any")

#Filter the data base: show data only on Thousand Barrels per day (kb/d)(KBD)
dayly_barrels = monthly.loc[monthly["UNIT_MEASURE"] == "KBD"]

#Filter the data base: show data only from countries we are interested in
exportadores = dayly_barrels.loc[dayly_barrels["REF_AREA"].isin(a_codes)]

#divide the data set in three areas of interest crude oil, ngl and total
crude_oil = exportadores.loc[exportadores["ENERGY_PRODUCT"] == "CRUDEOIL"]
NGL = exportadores.loc[exportadores["ENERGY_PRODUCT"] == "NGL"]
total_crude = exportadores.loc[exportadores["ENERGY_PRODUCT"] == "TOTCRUDE"]

In [61]:
#Add the full country name column
#Create a function who get the name from alpha 2 encode
def country(x):
    pais = pycountry.countries.get(alpha_2=x)
    return pais.name
#Apply that function to the column REF_AREA in order to get the country column
crude_oil["Country"] = crude_oil["REF_AREA"].apply(lambda x : country(x))

#Filter data to get exports and imports in different datasets
imports = crude_oil.loc[crude_oil["FLOW_BREAKDOWN"] == "TOTIMPSB"]
exports = crude_oil.loc[crude_oil["FLOW_BREAKDOWN"] == "TOTEXPSB"]

In [62]:
#Rename values column
imports = imports.rename(columns={"OBS_VALUE" : "Imports (kb/d)", "ASSESSMENT_CODE" : "Imp_notes"})
exports = exports.rename(columns={"OBS_VALUE" : "Exports (kb/d)", "ASSESSMENT_CODE" : "Exp_notes"})

#drop unecesary data columns 
imports = imports.drop(columns=["ENERGY_PRODUCT", "FLOW_BREAKDOWN", "UNIT_MEASURE", "REF_AREA"]) 
exports = exports.drop(columns=["ENERGY_PRODUCT", "FLOW_BREAKDOWN", "UNIT_MEASURE", "REF_AREA"])

In [65]:
#Merge both datasets in one
exp_netas = exports.merge(imports, how="left", on=["Country", "TIME_PERIOD"])
#Reorder the columns
exp_netas = exp_netas[['Country','TIME_PERIOD', 'Exports (kb/d)','Exp_notes','Imports (kb/d)', 'Imp_notes']]
#Create a column who substract imports from Exports
exp_netas["Net Exports"] = (exp_netas["Exports (kb/d)"] - exp_netas["Imports (kb/d)"])

In [70]:
exp_netas.dtypes

Country            object
TIME_PERIOD        object
Exports (kb/d)    float64
Exp_notes           int64
Imports (kb/d)    float64
Imp_notes           int64
Net Exports       float64
dtype: object

# NEXT STEPS
- TRANSFORMAR EL TIME PERIOD A datetime
- CREAR UNA TABLA EXTRA PARA ANÁLISIS VISUAL
- DEJAR LA TABLA ORIGINAL PARA DATA ANALISIS
- CREAR UNA GRÁFICA PARA ANALISIS REAL